# Lab 09: Variational Autoencoders

In this lab, we will learn how to build and train a variational autoencoder following [Kingma and Welling (2014)](https://arxiv.org/pdf/1312.6114.pdf).

The application the authors consider is a generative model for the MNIST digits dataset that uses a spherical Gaussian for the latent space.

We'll first look at the official VAE PyTorch example code (https://github.com/pytorch/examples/tree/master/vae).
They follow Kingma and Welling in that the encoder $q_\phi(\mathbf{x})$ and the decoder $p_\theta(\mathbf{z})$ are both fully-connected neural networks.
However, in the original paper, the authors use sigmoid hidden units and the AdaGrad optimizer. In the PyTorch VAE example, the implementers find that it
converges much faster with ReLU hidden units and the Adam optimizer.

After that, we'll explore the use of convolutional layers rather than fully-connected layers for the encoder and decoder, for MNIST, AIT ICT Faces, and CelebA.

## Deep generative models overview

Deep generative models have 2 major families:
 - Generative Adversarial Networks (GANs)
 - Variational Autoencoders (VAEs)

Both models can produce highly realistic content such as images, text documents, and sounds. The difference
is that GANs use two networks (generator and discriminator) that play a minimax game to optimize performance, whereas
the VAE is an autoencoder in which the distribution at the bottlneck layer is regularised during training in order to
ensure that the latent space has good properties (smoothness), enabling the decoder to act as a regularized generator
that can generate novel data consistent with the training data set. The term "variational" comes from the close relationship
between regularisation and the variational inference method in statistics.

Here's the idea of the GAN's structure from [Towards Data Science](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73):

<img src="img/gans.jpg" title="VAE" style="width: 640px;" />

and here's the idea of the VAE's structure (with convolutional encoder and decoder):

<img src="img/vae.jpg" title="VAE" style="width: 640px;" />


The encoder in an ordinary autoencoder maps the input to a low-dimensional repesentation at a "bottleneck" layer that preserves as much information as possible that will be
useful for reconstructing the input from the encoding. If we use a single fully connected linear layer as the bottleneck without any nonlinearity or other hidden layers,
use a single fully connected linear layer for the decoder, and
train the model to minimize L2 reconstruction error, it is a well-known result in the neural network literature that we obtain the same result as Principal Components
Analysis (PCA), a linear statistical dimensionality reduction technique more than 100 years old!

The autoencoding process, when thought of as a compression process, can be lossless or lossy:

<img src="img/encodedecode.png" title="Encoder-Decoder" style="width: 1080px;" />

## More on principal components analysis (PCA)

Assume $n_d$ is the dimensionality of the initial (decoded) space and that $n_e$ is the dimensionality of the representation at the bottleneck.
PCA constructs $n_e$ independent features as linear combinations of the input $n_d$ features. Since each feature is a linear combination of the input features,
we can envision the weights for each feature as a vector in the input space and the dot product operation as a projection operation. PCA will find the
projections of the data that are as well aligned as possible with the input data distribution. With L2 loss at the ouptut of the decoder,
we see that PCA is performing a type of linear regression. Minimizing reconstruction error given a fixed size representation at the bottleneck will require an
efficient representation at the bottleneck.

<img src="img/pca.png" title="PCA" style="width: 1080px;" />

In the case of linear dimensionality reduction, the optimal encoding is to take the $n_e$ eigenvectors of the data's covariance matrix corresponding to the
$n_e$ largest eigenvectors of that covariance matrix. The encoder's weight matrix is composed of these $n_e$ eignevectors, and the decoder's weight matrix
is composed of the same weight vector transposed:

<img src="img/eigen.png" title="Eigen" style="width: 1080px;" />

## Autoencoder

Now we extend the linear PCA concept to a situation in which we would like to obtain a more efficient representation at the bottleneck than can be obtained by PCA.
In place of linear projections, we have two neural networks acting as encoder and decoder. Data are fed to the autoencoder which compresses then decompresses the
input, and the decoded output is compared with the input. Errors are backpropagated. Hopefully, the encoder will find a representation at the bottleneck that can
be efficiently decoded to minimize reconstruction error.
The optimization problem looks like this:

<img src="img/dimen-reduc-eq.png" title="Equation" style="width: 250px;" />

where

<img src="img/where.png" title="Equation" style="width: 100px;" />

is L2 loss:

<img src="img/autoencoder.png" title="Autoencoder" style="width: 640px;" />

Ideally, we will obtain an extremely efficient encoding that is also
meaningful for the input data set:

<img src="img/autoencoderwithpca.png" title="Object classifier" style="width: 1080;" />


<img src="img/autoencoderconcept.png" title="Autoencoder concept" style="width: 1080px;" />

## Variational autoencoders (VAEs)

Now, let's discuss how the VAE overcomes some issues with the ordinariy autoencoder.
A VAE is trained with regularization to avoid overfitting and ensure that the latent space
has good properties such as smoothness that encourage generation of novel samples consistent
with the input distriution. Instead of encoding an input as a single point in the latent space
each input is mapped to a *distribution* over the latent space. The model is then trained as follows:

1. The input is encoded, resulting in a distribution over the latent space
2. A point from the latent space is sampled from that distribution
3. The sampled point is decoded and the reconstruction error is computed
4. The reconstruction error is backpropagated through the network

Here's the idea, visually:

<img src="img/diffAE_VAEs.png" title="Difference between autoencoder and VAEs" style="width: 1080;" />

We would ordinarily (but not necessarily) choose the encoded distribution to be multivariate Gaussian, meaning the encoder should produce a mean vector and covariance
matrix for each input. Putting together the output loss function (reconstruction error) with the regularization at the latent layer will give us an encoder and decoder
that are both performant in terms of reconstruction error and well organized at the latent layer. Since we have to reconstruct the input accurately from any of the
samples it might generate at the latent layer, we should obtain smooth behavior in which interpolation of latent vectors yield realistic and smoothly varying samples
at the output. Obtaining this result means "simply" penalizing the Kulback-Leibler divergence between the latent distribution and a standardized Gaussian.

<img src="img/VAEs.png" title="VAEs loss function" style="width: 640;" />

Note that we cannot push the encoded distribution for every input to the standard normal distribution exactly, as we would not then be able to reconstruct the input!
But simply penalizing the KL divergence on each iteration will tend to move the marginal distribution over the latent space for the whole training set
toward a well organized normal distribution.

## Summary of VAEs

OK, let's summarize. The encoder maps the input to the parameters of a distribution:

<img src="img/encoderVAE.png" title="Encoder of VAEs" style="width: 640" />

The decoder maps an element of the latent space to an element of the data space:

<img src="img/decoderVAE.png" title="Decoder of VAEs" style="width: 640" />

The reparameterization trick introduced by the authors
isolates what we cannot backpropagate through (sampling from a Gaussian with a particular mean and covariance) from
the production of the parameters of the distribution, which can be backpropagated through:

<img src="img/backpropVAE.png" title="Trick" style="width: 640" />

Putting this together we have the full VAE in all of its glory:

<img src="img/FullVAE.png" title="Full VAE" style="width: 640" />

## VAE example
Here is a slightly modified version of the PyTorch VAE example. Try it out and take a look at the results.

In [3]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [4]:
log_interval = 100
seed = 1

torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MNIST Dataset

In [6]:
out_dir = './torch_data/VGAN/MNIST/dataset' #you can use old downloaded dataset, I use from VGAN
batch_size=128

train_loader = torch.utils.data.DataLoader(datasets.MNIST(root=out_dir, download=True, train=True, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST(root=out_dir, train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)

# Create class VAE

In [7]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # for encoder
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)

        # for decoder
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        # 0.5 for square root (variance to standard deviation)
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Loss function for VAE

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

# Train function

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

# Test function

In [8]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

# Create the VAE model and optimizer

In [9]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [10]:
epochs = 50
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 20).to(device)
        sample = model.decode(sample).cpu()
        print("save image: " + 'results/sample_' + str(epoch) + '.png')
        save_image(sample.view(64, 1, 28, 28), 'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 550.595764
Train Epoch: 1 [12800/60000 (21%)]	Loss: 186.938980
Train Epoch: 1 [25600/60000 (43%)]	Loss: 154.969391
Train Epoch: 1 [38400/60000 (64%)]	Loss: 136.863266
Train Epoch: 1 [51200/60000 (85%)]	Loss: 132.102951
====> Epoch: 1 Average loss: 164.8554
====> Test set loss: 128.1345
save image: results/sample_1.png
Train Epoch: 2 [0/60000 (0%)]	Loss: 123.857475
Train Epoch: 2 [12800/60000 (21%)]	Loss: 125.330246
Train Epoch: 2 [25600/60000 (43%)]	Loss: 118.836884
Train Epoch: 2 [38400/60000 (64%)]	Loss: 120.265221
Train Epoch: 2 [51200/60000 (85%)]	Loss: 121.937012
====> Epoch: 2 Average loss: 122.0223
====> Test set loss: 116.0039
save image: results/sample_2.png
Train Epoch: 3 [0/60000 (0%)]	Loss: 117.034286
Train Epoch: 3 [12800/60000 (21%)]	Loss: 116.452927
Train Epoch: 3 [25600/60000 (43%)]	Loss: 118.498375
Train Epoch: 3 [38400/60000 (64%)]	Loss: 116.039642
Train Epoch: 3 [51200/60000 (85%)]	Loss: 114.187653
====> Epoch: 3 Average loss: 114.

Here is the result after 1 epochs

<img src="img/sample_1.png" title="1 epoch" style="width: 640" />

And here is the result after 50 epochs

<img src="img/sample_50.png" title="50 epochs" style="width: 640" />

Last year
## In-lab exercise
Set up visdom to visualize the progress of train and test loss during training, and also visualize the test reconstructions and reconstructed samples from the latent space.

Experiment with number of epochs, batch size, learning rate, etc. to get the best results you can.

## In-lab/at home exercise
Repeat the experiment with MNIST, AIT ICT Faces, CelebA. Use convolutional layers rather than fully connected layers for the encoder and decoder, keeping the prior over the latent representation as a spherical Gaussian.